# Net Flow EDA (30-min buckets)

This notebook explores station-level net bike flow aggregated in 30 min intervals. The aim is to understand imbalance behaviour and define shortage/surplus thresholds for downstream rebalancing decisions.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('processed_data/net_flow_30min')
df.head()

,station_id,time_bucket,net_flow
0,001072,2024-01-25 08:00:00,-87
1,001072,2025-01-09 08:00:00,-82
2,001072,2023-12-13 08:00:00,-78
3,001072,2023-02-23 08:00:00,-78
4,001072,2025-05-22 08:00:00,-78


In [3]:
rows, cols = df.shape 
print(f'Rows = {rows}')
print(f'Columns = {cols}')

Rows = 17759058
Columns = 3


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17759058 entries, 0 to 17759057
Data columns (total 3 columns):
 #   Column       Dtype         
---  ------       -----         
 0   station_id   object        
 1   time_bucket  datetime64[us]
 2   net_flow     int64         
dtypes: datetime64[us](1), int64(1), object(1)
memory usage: 406.5+ MB


In [5]:
df['station_id'].nunique()

1244

In [6]:
station_counts = (
    df.groupby("station_id")["time_bucket"]
      .nunique()
      .sort_values(ascending=False)
)

station_counts.describe()


count     1244.000000
mean     14275.770096
std      10392.361544
min          2.000000
25%       1721.750000
50%      17348.000000
75%      23337.000000
max      34776.000000
Name: time_bucket, dtype: float64

In [ ]:
station_counts.head(10) #these are real stations


station_id
300083    34776
300031    34101
200025    33708
200183    33319
300086    33221
200230    32957
000960    32941
200250    32922
001161    32548
200246    31901
Name: time_bucket, dtype: int64

In [ ]:
station_counts.tail(10) #junk stations

station_id
3429444       354
1203          276
001219444     236
200032444     191
200217old2    142
022168         28
001057_old     20
123456          5
300254          2
010626          2
Name: time_bucket, dtype: int64

- Chose cutoff of 500 so any station that has less than 500 buckets are categorised as junk stations and removed

In [10]:
min_buckets = 500
station_counts = (
    df.groupby("station_id")["time_bucket"]
      .nunique()
)

valid_stations = station_counts[station_counts >= 500].index

df = df[df["station_id"].isin(valid_stations)]

df["station_id"].nunique()

1232

- Filter reduced no. of unique stations from 1244 to 1232
- 1232 stations were operational at any point in the dataset. 

In [11]:
df.groupby("station_id")["time_bucket"].count().describe()


count     1232.000000
mean     14413.120130
std      10348.743129
min        500.000000
25%       1739.750000
50%      17470.000000
75%      23382.500000
max      34776.000000
Name: time_bucket, dtype: float64

This tell us:
- 500 buckets = 10.4 days apprx
- 17,470 buckets = 364 days apprx
- 34776 buckets = 724 days (2 years) apprx 

So:
- Worst stations still has > 10 days of data
- Typical station has apprx 1 year
- Core stations have apprx 2 years

In [12]:
df_clean = df.copy()

In [15]:
#save data 
from pathlib import Path 

out_dir = Path('processed_data')
out_dir.mkdir(exist_ok = True)

df_clean.to_parquet(
    out_dir / 'net_flow_30min_clean.parquet',
    index = False
)

In [17]:
#verify correct table saved
test = pd.read_parquet('processed_data/net_flow_30min_clean.parquet')

test.shape
print(test['station_id'].nunique())
test['time_bucket'].min(), test['time_bucket'].max()

1232


(Timestamp('2022-09-12 05:00:00'), Timestamp('2025-06-01 01:00:00'))